In [ ]:
# ---------------------------------------------------------
# Seção 1: Importar biblioteca e conectar ao banco de dados
import sqlite3

# Conecta a um banco em memória (não salva arquivo físico, ideal para testes)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Habilita o uso de chaves estrangeiras no SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

print("Conexão com o banco estabelecida e PRAGMA configurado.")

# ---------------------------------------------------------
# Seção 2: Criação das tabelas

# Tabela pessoas
cursor.execute('''
CREATE TABLE IF NOT EXISTS pessoas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    idade INTEGER
);
''')

# Tabela produtos
cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descricao TEXT NOT NULL,
    estoque INTEGER,
    preco REAL
);
''')

# Tabela vendas com chaves estrangeiras para pessoas e produtos
cursor.execute('''
CREATE TABLE IF NOT EXISTS vendas (
    id_venda INTEGER PRIMARY KEY AUTOINCREMENT,
    id_pessoa INTEGER,
    id_produto INTEGER,
    quantidade INTEGER,
    FOREIGN KEY (id_pessoa) REFERENCES pessoas(id),
    FOREIGN KEY (id_produto) REFERENCES produtos(id)
);
''')

conn.commit()
print("Tabelas criadas com sucesso.")

# ---------------------------------------------------------
# Seção 3: Inserção de dados nas tabelas

# Inserindo pessoas
pessoas = [
    ('Maria', 30),
    ('João', 25),
    ('Ana', 40)
]

cursor.executemany("INSERT INTO pessoas (nome, idade) VALUES (?, ?);", pessoas)

# Inserindo produtos
produtos = [
    ('Caneta', 100, 1.50),
    ('Caderno', 50, 6.50),
    ('Borracha', 75, 0.80)
]

cursor.executemany("INSERT INTO produtos (descricao, estoque, preco) VALUES (?, ?, ?);", produtos)

conn.commit()
print("Dados inseridos em pessoas e produtos.")

# ---------------------------------------------------------
# Seção 4: Inserção de vendas e consultas avançadas (JOIN)

# Inserir algumas vendas
vendas = [
    (1, 1, 10),  # Maria comprou 10 canetas
    (2, 2, 2),   # João comprou 2 cadernos
    (3, 3, 5)    # Ana comprou 5 borrachas
]

cursor.executemany("INSERT INTO vendas (id_pessoa, id_produto, quantidade) VALUES (?, ?, ?);", vendas)
conn.commit()
print("Vendas inseridas.")

# Consulta com JOIN para mostrar detalhes da venda
query = '''
SELECT v.id_venda, p.nome, pr.descricao, v.quantidade, pr.preco, (v.quantidade * pr.preco) AS total
FROM vendas v
JOIN pessoas p ON v.id_pessoa = p.id
JOIN produtos pr ON v.id_produto = pr.id
ORDER BY total DESC;
'''

cursor.execute(query)
results = cursor.fetchall()

print("\nResumo das vendas:")
for row in results:
    print(f"Venda ID: {row[0]}, Cliente: {row[1]}, Produto: {row[2]}, Quantidade: {row[3]}, Preço unitário: R${row[4]:.2f}, Total: R${row[5]:.2f}")

# ---------------------------------------------------------
# Seção 5: Consultas extras (exemplos de WHERE e ORDER BY)

# Produtos com estoque menor que 80 unidades
cursor.execute("SELECT * FROM produtos WHERE estoque < 80 ORDER BY estoque;")
produtos_baixo_estoque = cursor.fetchall()

print("\nProdutos com estoque menor que 80:")
for produto in produtos_baixo_estoque:
    print(produto)

# Pessoas com idade maior que 28
cursor.execute("SELECT * FROM pessoas WHERE idade > 28 ORDER BY idade DESC;")
pessoas_mais_velhas = cursor.fetchall()

print("\nPessoas com idade maior que 28:")
for pessoa in pessoas_mais_velhas:
    print(pessoa)

# ---------------------------------------------------------
# Seção 6: Encerramento da conexão

conn.close()
print("\nConexão com o banco encerrada.")
